# Import

In [ ]:
import numpy as np 
from os import path
import pandas as pd 
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import rcParams
tickfontsize=20
labelfontsize = tickfontsize

In [ ]:
import datetime

In [ ]:
import math

In [ ]:
import time

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Load data

In [ ]:
ml_data = pd.read_csv('~/efrc/prep_data/no_cat_v1/data_DONOTOUCH/ml_data.csv')

In [ ]:
ml_data.head()

In [ ]:
total_frac = 1
start_str = 'SMILES'
end_str = 'valence_pa'

In [ ]:
fp_dat = ml_data.sample(frac=total_frac, random_state=0)
fp_dat

In [ ]:
def eq_space(x, y, n, force_int=False):
    step = (y - x) / (n - 1)
    if force_int:
        return [int(x + step * i) for i in range(n)]
    return [x + step * i for i in range(n)]

In [ ]:
#define default params
defaults = {"patience":10, "training_pct":.8, "n_layer":2, "n_unit":10, "activation":'relu', "loss":'mse', 
            "opt":'adam', "val_pct":.2} #patience, training fraction, n hidden layers, n hidden units, activation, loss, optimizer, validation split

In [ ]:
#define initial grid
init_grid = {"patience":eq_space(20, 1000, 5, True), "training_pct":eq_space(.5, .8, 5), 
             "n_layer":eq_space(3, 20, 5, True), "n_unit":eq_space(20, 1000, 5, True), "activation":['relu', 'tanh', 'sigmoid'],
             "loss":['huber_loss', 'mse', 'mean_absolute_error', 'logcosh'], 
            "opt":['sgd', 'rmsprop', 'adamax', 'adam', 'adagrad'], "val_pct":[.3, .5, 5]}
#patience, training fraction, n hidden layers, n hidden units, activation, loss, optimizer, validation split

In [ ]:
property_used = 'norm_CH4_v/v_1_bar' #column name of target

In [ ]:
fp_dat = ml_data.sample(frac=total_frac, random_state=0)

In [ ]:
def getData(ml_data, total_frac, start_str, end_str, training_pct, batch_size=1, norm=True):
    '''
    get normalized training and test data
    '''
    fp_dat = ml_data.sample(frac=total_frac, random_state=0)
    train_dataset = fp_dat.sample(frac=training_pct,random_state=2)
    test_dataset = fp_dat.drop(train_dataset.index)
    train_label = train_dataset[property_used]
    test_label = test_dataset[property_used]
    for ind, col in enumerate(ml_data.columns):
        if start_str in col:
            start_col = ind + 1
        elif end_str == col:
            end_col = ind


    features = list(ml_data.columns[start_col:end_col])
    other_props = ['norm_Dom._Pore_(ang.)',
     'norm_Max._Pore_(ang.)',
     'norm_Void_Fraction',
     'norm_Surf._Area_(m2/g)',
     'norm_Vol._Surf._Area',
     'norm_Density',
      'norm_valence_pa',
       'norm_atomic_rad_pa_(angstroms)',
         'norm_affinity_pa_(eV)',
           'norm_ionization_potential_pa_(eV)',
               'norm_electronegativity_pa']

    features = features + other_props

    train_fp = train_dataset[features]
    test_fp = test_dataset[features]
    
    if norm:
        # Summary of training ( and test)
        train_stats = train_fp.describe()
        train_stats = train_stats.transpose()

        test_stats = test_fp.describe()
        test_stats = test_stats.transpose()
        ######################################
        
        # Remove features with 0 std
        my_set ={}
        my_set.update(train_stats['std'][train_stats['std'] == 0])
        my_set.update(test_stats['std'][test_stats['std'] == 0])


        train_fp1 = train_fp.drop(my_set.keys(), axis=1)

        test_fp1 = test_fp.drop(my_set.keys(), axis=1)
        ###################################################
        
        # Normalization (check it this is required. Try without first)
        def norm(x):
            stats = x.describe()
            stats = stats.transpose()
            return (x - stats['mean']) / stats['std']

        normed_train_fp = norm(train_fp1)
        normed_test_fp = norm(test_fp1)
        train_fp = normed_train_fp
        test_fp = normed_test_fp
    
    train_data = tf.data.Dataset.from_tensor_slices((train_fp.to_numpy().astype(np.float32), 
                                                     train_label.to_numpy().astype(np.float32))).batch(batch_size)
    test_data = tf.data.Dataset.from_tensor_slices((test_fp.to_numpy().astype(np.float32), 
                                                    test_label.to_numpy().astype(np.float32))).batch(batch_size)

    return train_data, test_data
    #model.fit(train_data, validation_data=train_data, epochs=10)

In [ ]:
# define the keras model
def buildModel1(train_data, n_layer, n_unit, activation, loss, opt, dropout=False):
    
    #n_col = train_fp.shape[1]
    n_col = train_data.element_spec[0].shape[1]
    print(n_col)
#     model_guts = [layers.Dense(n_unit, activation=activation, 
#                                input_shape=[n_col])] + [layers.Dense(n_unit, 
#                                 activation=activation) for i in range(n_layer - 1)] + [layers.Dense(1, activation='linear')]
    model_guts = []
    model_guts.append(layers.Dense(n_unit, activation=activation, 
                                input_shape=[n_col]))
    for i in range(n_layer - 1):
        model_guts.append(layers.Dense(n_unit, activation=activation))
        if dropout:
            model_guts.append(layers.Dropout(.3))
    
    model_guts.append(layers.Dense(1, activation='linear'))
    
    model = keras.Sequential(model_guts)

    
    model.compile(loss=loss,
        optimizer=opt,
        metrics=['mae', 'mse'])
    
    model.summary()
    
    return model

In [ ]:
def trainModel(model, patience, val_pct, train_data):
    EPOCHS = 2000
    # The patience parameter is the amount of epochs to check for improvement
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)
    checkpoint_callbacks = keras.callbacks.ModelCheckpoint(filepath='model_checkpoint.h5', monitor='val_loss',\
                                                          verbose=1, save_best_only=True, mode='min')
    # early_history = model.fit(normed_train_data, train_label.to_numpy(), 
    #                     epochs=EPOCHS, validation_split = 0.2, verbose=1, callbacks=[early_stop,checkpoint_callbacks])
    log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    early_history = model.fit(train_data,
                        epochs=EPOCHS, validation_data = train_data, verbose=1,\
                              callbacks=[early_stop,checkpoint_callbacks,tfdocs.modeling.EpochDots(),tensorboard_callback])

In [ ]:
def evaluate_model(ml_data, total_frac, start_str, end_str, patience, training_pct, n_layer, n_unit, activation, 
                   loss, opt, val_pct, batch_size=10, norm=True):
    '''
    This function creates a model and returns its mse
    '''
    print("Patience: ", patience)
    print("training_pct: ", training_pct)
    print("n_layer: ", n_layer)
    print("n_unit: ", n_unit)
    print("activation: ", activation)
    print("loss: ", loss)
    print("opt: ", opt)
    print("val_pct: ", val_pct)
    train_data, test_data = getData(ml_data, total_frac, start_str, 
                                                                           end_str, 
                                    training_pct, batch_size=batch_size, norm=norm) #get normalized training and test data
    model = buildModel1(train_data, n_layer, n_unit, activation, loss, opt)
    
    
    trainModel(model, patience, val_pct, train_data)
    
    loss, mae, mse = model.evaluate(test_data, verbose=2)
    
    return mse, model

In [ ]:
def varyParams(ml_data, default_params, grid, total_frac, start_str, end_str):
    par_d = {}
    for grid_k,grid_v in zip(grid.keys(), grid.values()):
        for val in grid_v:
            for def_k,def_v in zip(default_params.keys(), default_params.values()):
                exec(def_k+"=def_v")
                exec("global " + def_k)
            
            exec(grid_k+"=val")
            exec("global " + grid_k)
            l = list(default_params.keys())
            mse = eval('evaluate_model(ml_data, total_frac, start_str, end_str, ' + l[0] + ',' + l[1] + ',' + l[2] + ',' + l[3] + ',' + l[4] + ',' + l[5] + ',' + l[6] + ',' + l[7] + ')')
            par_d[val] = mse

    #r = eval('[' + l[0] + '_d' + ', ' + l[1] + '_d' + ', ' + l[2] + '_d' + ', ' + l[3] + '_d' + ', ' + l[4] + '_d' + ', ' + l[5] + '_d' + ', ' + l[6] + '_d' + ', ' + l[7] + '_d' + ']')
    return par_d
    

In [ ]:
train_dataset = fp_dat.sample(frac=training_pct,random_state=2)
test_dataset = fp_dat.drop(train_dataset.index)

In [ ]:
train_label = train_dataset[property_used]
test_label = test_dataset[property_used]

In [ ]:
for ind, col in enumerate(ml_data.columns):
    if start_str in col:
        start_col = ind + 1
    elif end_str == col:
        end_col = ind


features = list(ml_data.columns[start_col:end_col])
other_props = ['norm_Dom._Pore_(ang.)',
 'norm_Max._Pore_(ang.)',
 'norm_Void_Fraction',
 'norm_Surf._Area_(m2/g)',
 'norm_Vol._Surf._Area',
 'norm_Density',
  'norm_valence_pa',
   'norm_atomic_rad_pa_(angstroms)',
     'norm_affinity_pa_(eV)',
       'norm_ionization_potential_pa_(eV)',
           'norm_electronegativity_pa']

features = features + other_props

In [ ]:
len(features)

In [ ]:
train_fp = train_dataset[features]
test_fp = test_dataset[features]

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((train_fp.to_numpy().astype(np.float32), 
                                                     train_label.to_numpy().astype(np.float32))).batch(10000)

In [ ]:
test_data = tf.data.Dataset.from_tensor_slices((test_fp.to_numpy().astype(np.float32), 
                                                    test_label.to_numpy().astype(np.float32))).batch(10000)

In [ ]:
# plot of test and training sets
fig,ax = plt.subplots(figsize = (8,5))
n_bins=30
n, bins, patches = plt.hist(train_label, n_bins, normed=0, lw=0.5, edgecolor='k', facecolor='#FDA65F', alpha=1,label = 'Training set')
n, bins, patches = plt.hist(test_label, n_bins, normed=0, lw=0.5, edgecolor='k', facecolor='green', alpha=1, label = 'Test set')
plt.xlabel('y_val',fontsize=labelfontsize)
plt.ylabel('Count',fontsize=labelfontsize)
#ax.set_xlim(2,12)
ax.legend()
fig.tight_layout()
plt.savefig('%s.png'%property_used,dpi=200)


In [ ]:
# Summary of training ( and test)
train_stats = train_fp.describe()
train_stats = train_stats.transpose()
train_stats

In [ ]:
test_stats = test_fp.describe()
test_stats = test_stats.transpose()
test_stats

In [ ]:
train_stats['std'][train_stats['std'] == 0]

In [ ]:
test_stats['std'][test_stats['std'] == 0].keys()

In [ ]:
my_set ={}
my_set.update(train_stats['std'][train_stats['std'] == 0])
my_set.update(test_stats['std'][test_stats['std'] == 0])

In [ ]:
my_set.keys()

In [ ]:
train_fp1 = train_fp.drop(my_set.keys(), axis=1)
#test_fp1 = test_fp.drop(train_stats['std'][train_stats['std'] == 0].index, axis=1)

In [ ]:
test_fp1 = test_fp.drop(my_set.keys(), axis=1)

In [ ]:
# Normalization (check it this is required. Try without first)
def norm(x):
    stats = x.describe()
    stats = stats.transpose()
    return (x - stats['mean']) / stats['std']

In [ ]:
normed_train_data = norm(train_fp1)
normed_test_data = norm(test_fp1)

In [ ]:
normed_test_data.head()

In [ ]:
normed_train_data.head()

In [ ]:
len(train_fp.keys())

In [ ]:
# define the keras model
def build_model():
    model = keras.Sequential([
        layers.Dense(100, activation='relu', input_shape=[len(train_fp.keys())]),
        layers.Dense(100, activation='relu'),
        layers.Dense(100, activation='relu'),
        layers.Dense(1)
    ])

    model.compile(loss='mse',
        optimizer='adam',
        metrics=['mae', 'mse'])
    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
# NN model training
EPOCHS = 5000

model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)
checkpoint_callbacks = keras.callbacks.ModelCheckpoint(filepath='model_checkpoint.h5', monitor='val_loss',\
                                                      verbose=1, save_best_only=True, mode='min')
# early_history = model.fit(normed_train_data, train_label.to_numpy(), 
#                     epochs=EPOCHS, validation_split = 0.2, verbose=1, callbacks=[early_stop,checkpoint_callbacks])
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_history = model.fit(train_fp.to_numpy(), train_label.to_numpy(), 
                    epochs=EPOCHS, validation_split = 0.2, verbose=1,\
                          callbacks=[early_stop,checkpoint_callbacks,tfdocs.modeling.EpochDots(),tensorboard_callback])
                        


In [ ]:
# Check if run converged
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
plotter.plot({'Early Stopping': early_history}, metric = "mae")
#plt.ylim([0, 0.15])
plt.ylabel('MAE')

In [ ]:
# Evaluation of test error and plotting parity

#model = tf.keras.models.load_model('model_checkpoint_bandgap.h5')
loss, mae, mse = model.evaluate(test_fp.to_numpy(), test_label.to_numpy(), verbose=2)
print("Testing set Mean Abs Error: {:5.2f} bg".format(mae))

tr_loss, tr_mae, tr_mse = model.evaluate(train_fp.to_numpy(), train_label.to_numpy(), verbose=2)

tr_rmse = math.sqrt(tr_mse)
rmse = math.sqrt(mse)

test_predictions = model.predict(test_fp.to_numpy()).flatten()


train_predictions = model.predict(train_fp.to_numpy()).flatten()

fig1,ax1 = plt.subplots(figsize = (8,8))
ax1.scatter(test_label, test_predictions, c='r',s=30)

ax1.scatter(train_label, train_predictions, c='b',s=30)

ax1.set_xlabel('True normalized CH4 Uptake @ 1 bar',fontsize=labelfontsize)
ax1.set_ylabel('Predicted normalized CH4 Uptake @ 1 bar',fontsize=labelfontsize)
ax1.set_xlim(min([min(test_label),min(test_predictions)])-1,max([max(test_label),max(test_predictions)])+1)
ax1.set_ylim(min([min(test_label),min(test_predictions)])-1,max([max(test_label),max(test_predictions)])+1)
ax1.legend()
plot_x_min, plot_x_max = plt.xlim()
plot_y_min, plot_y_max = plt.ylim()

ax1.plot(np.linspace(plot_x_min,plot_x_max,100),np.linspace(plot_y_min,plot_y_max,100),c='k',ls='--')
text_position_x = plot_x_min + (plot_x_max - plot_x_min) * 0.05
text_position_y = plot_y_max - (plot_y_max - plot_y_min) * 0.15

ax1.text(text_position_x, text_position_y, "RMSE test=" + str("%.4f" % rmse) + '\n' + 
         "RMSE train=" + str("%.4f" % tr_rmse), ha='left', fontsize=16)

# ax1.text(text_position_x, text_position_y, "MAE=" + str("%.4f" % mae) + ' \n' + 
#          "MSE=" + str("%.4f" % mse), ha='left', fontsize=16)
fig.tight_layout()
plt.savefig('./%s_test_parity_%s.png'%(property_used, total_frac),dpi=200)



In [ ]:
error = test_predictions - test_label
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error")
_ = plt.ylabel("Count")

# Test functions

In [ ]:
eq_space(4,9, 5, True)

In [ ]:
total_frac = 1
defaults = [10, .8, 3, 100, 'relu', 'mse', 'adam', .2]
patience = defaults[0]
training_pct = defaults[1]
n_layer = defaults[2]
n_unit = defaults[3]
activation = defaults[4]
loss = defaults[5]
opt = defaults[6]
val_pct = defaults[7]

In [ ]:
mse = evaluate_model(ml_data, total_frac, start_str, end_str, patience, training_pct, n_layer, n_unit, activation, 
                   loss, opt, val_pct)

In [ ]:
mse

In [ ]:
total_frac = .1
defaults = {"patience":10, "training_pct":.8, "n_layer":7, "n_unit":20, "activation":'relu', "loss":'mean_absolute_error', 
            "opt":'rmsprop', "val_pct":.2}
all_grid = {"patience":[10], "training_pct":eq_space(.5, .8, 5), 
             "n_layer":eq_space(3, 20, 5, True), "n_unit":eq_space(20, 1000, 5, True), "activation":['relu', 'tanh', 'sigmoid'],
             "loss":['huber_loss', 'mse', 'mean_absolute_error', 'logcosh'], 
            "opt":['sgd', 'rmsprop', 'adamax', 'adam', 'adagrad'], "val_pct":eq_space(.2, .5, 5)}


init_grid = {"val_pct":eq_space(.2, .5, 5)}

In [ ]:
#patience_d, training_pct_d, n_layer_d, n_unit_d, activation_d, loss_d, opt_d, val_pct_d 
r = varyParams(ml_data, defaults, init_grid, total_frac, start_str, end_str)

In [ ]:
r

In [ ]:
start = time.time()

training_pct = .9
total_frac = 1
mse, model = evaluate_model(ml_data, total_frac, start_str, end_str, 10, training_pct, 3, 200, 'relu', 
                   'mse', 'adam', .2, batch_size=5000, norm=False)

end = time.time()

print("Time elapsed: ", end - start)

In [ ]:
mse

In [ ]:
tr_loss, tr_mae, tr_mse = model.evaluate(train_data, verbose=2)

In [ ]:
tr_mse

In [ ]:
math.sqrt(mse)

In [ ]:
preds = model.predict(test_data)

In [ ]:
len(preds)

In [ ]:
preds.size

In [ ]:
len(test_label.values)

In [ ]:
plt.scatter(preds, test_label)